In [1]:
import sqlite3
from database_handler import ferGroupS
from tqdm import tqdm

In [2]:
# Connect to the SQLite database
conn = sqlite3.connect('databases/graphs.db')
cursor = conn.cursor()

# Create checkpoint table with a primary key to ensure only one row exists
cursor.execute("""
CREATE TABLE IF NOT EXISTS checkpoint (
    id INTEGER PRIMARY KEY,
    last_id INTEGER
)
""")
conn.commit()

# Fetch the last processed ID from the checkpoint table
cursor.execute("SELECT last_id FROM checkpoint WHERE id = 1")
row = cursor.fetchone()
last_id = row[0] if row else 0

print("Resuming from ID:", last_id)

# Get the total number of rows to process
cursor.execute("SELECT COUNT(*) FROM graphs WHERE id > ?", (1,))
total_rows = cursor.fetchone()[0]

# Fetch all rows into memory starting from last processed ID
cursor.execute("SELECT id, graph6 FROM graphs WHERE id > ? ORDER BY id", (last_id,))
rows = cursor.fetchall()

progress_bar = tqdm(total=total_rows, initial=0, desc="Processing Rows")

# Process each row
for row in rows:
    current_id, graph6 = row

    fer_group_s = ferGroupS(graph6)

    # Update the row with the computed result
    cursor.execute("UPDATE graphs SET fer_group = ? WHERE id = ?", (fer_group_s, current_id))
    
    # Update the checkpoint and commit immediately
    cursor.execute("INSERT OR REPLACE INTO checkpoint (id, last_id) VALUES (1, ?)", (current_id,))
    conn.commit()  # Ensure checkpoint and data are saved after each row  

    # Update the progress bar
    progress_bar.update(1)

# Close the progress bar and connection
progress_bar.close()
conn.close()

Resuming from ID: 38844


Processing Rows:  25%|██▍       | 12891/51734 [41:29:10<125:00:20, 11.59s/it]
